In [40]:
from math import *

import pandas as pd 
import sklearn
import numpy as np
import os 
import matplotlib.pyplot as plt


In [26]:
try :
    os.chdir("data")
except:
    print(os.getcwd())

c:\Users\Utlisateur\Documents\CentraleSupelec\3A\IA\ApprAuto\Projet ML bis\Airbnb_berlin_price_prediction\data


In [27]:
Trainset = pd.read_csv("train_airbnb_berlin.csv")
X_train=Trainset.drop('Price',axis=1)
X_test=Trainset['Price']

In [28]:
X_train.columns

Index(['Listing ID', 'Listing Name', 'Host ID', 'Host Name', 'Host Since',
       'Host Response Time', 'Host Response Rate', 'Is Superhost',
       'neighbourhood', 'Neighborhood Group', 'City', 'Postal Code',
       'Country Code', 'Country', 'Latitude', 'Longitude', 'Is Exact Location',
       'Property Type', 'Room Type', 'Accomodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Square Feet', 'Guests Included', 'Min Nights', 'Reviews',
       'First Review', 'Last Review', 'Overall Rating', 'Accuracy Rating',
       'Cleanliness Rating', 'Checkin Rating', 'Communication Rating',
       'Location Rating', 'Value Rating', 'Instant Bookable',
       'Business Travel Ready'],
      dtype='object')

In [29]:
#### Preprocessing
to_drop=['Listing ID', 'Listing Name', "Host ID","City",'Country Code','Country','Square Feet' ]
X_train_dropped=X_train.drop(to_drop,axis=1)

X_train_dropped


,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,Postal Code,Latitude,Longitude,...,Last Review,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready
0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,10437.0,52.54652,13.41792,...,2017-08-08,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f
1,Dulie,2013-04-07,NaN,NaN,f,Pankow,Pankow,13187.0,52.56512,13.42214,...,2019-04-30,90.0,9.0,9.0,10.0,10.0,9.0,10.0,f,f
2,Geank,2016-02-07,NaN,NaN,f,Prenzlauer Berg,Pankow,10439.0,52.54741,13.42521,...,2016-07-04,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f
3,Alix,2018-06-26,NaN,NaN,f,Friedrichshain,Friedrichshain-Kreuzberg,10245,52.50958,13.45144,...,2018-08-12,100.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f
4,Lurina,2013-05-16,within a few hours,92%,t,Prenzlauer Berg,Pankow,10405.0,52.52995,13.41558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15687,Ascharl,2013-06-22,a few days or more,33%,f,Kreuzberg,Friedrichshain-Kreuzberg,10961.0,52.49016,13.39241,...,2019-01-02,98.0,10.0,9.0,10.0,10.0,10.0,10.0,f,f
15688,Marki,2015-04-19,NaN,NaN,f,NeukÃ¶lln,NeukÃ¶lln,12055.0,52.47586,13.45362,...,2016-07-31,100.0,10.0,9.0,10.0,10.0,10.0,9.0,f,f
15689,Ada,2016-08-11,NaN,NaN,f,Wedding,Mitte,13351.0,52.54196,13.34179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f
15690,Lanna,2015-12-06,within a day,50%,f,SchÃ¶neberg,Tempelhof - SchÃ¶neberg,12159.0,52.47179,13.33103,...,2018-08-07,90.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f


In [48]:
def check_nan(x):
    try:
        return isnan(x)
    except:
        return x


( X_train_dropped.iloc[2]).apply(check_nan)



Host Name                          Geank
Host Since                    2016-02-07
Host Response Time                  True
Host Response Rate                  True
Is Superhost                           f
neighbourhood            Prenzlauer Berg
Neighborhood Group                Pankow
Postal Code                      10439.0
Latitude                           False
Longitude                          False
Is Exact Location                      t
Property Type                  Apartment
Room Type                Entire home/apt
Accomodates                            3
Bathrooms                            1.0
Bedrooms                             1.0
Beds                                 2.0
Guests Included                        1
Min Nights                             1
Reviews                            False
First Review                  2016-04-19
Last Review                   2016-07-04
Overall Rating                     False
Accuracy Rating                    False
Cleanliness Rati

In [81]:
def switch_list_bool(list_bool):
    list_bool_inverse=[]
    for a in (list_bool):
        list_bool_inverse.append(bool(1-a))
    return list_bool_inverse


def list_star_in_raws(df):
    res=[]
    for i in range(len(df)):
        raw=df.iloc[i]
        res.append("*" not in raw.values)
    return res

def list_nan_in_raws(df):
    list_bool_nan=[]
    for i in range(len(df)):
        raw=df.iloc[i].apply(check_nan)
        list_bool_nan.append(True not in raw.values)
    return list_bool_nan

In [79]:
list_bool_star=list_star_in_raws(X_train_dropped)
list_bool_nan=list_nan_in_raws(X_train_dropped.drop(["Host Response Time","Host Response Rate"],axis=1))
print(15962-sum(list_bool_star))
print(15962-sum(list_bool_nan))

True

In [75]:
print("shape X_train_Drop (X_d): ", X_train_dropped.shape)
print("shape X_d sans NaN : ",X_train_dropped.dropna().shape)
print("shape X_d sans colonnes 2 et 3 ET sans NaN : ",X_train_dropped.drop(["Host Response Time","Host Response Rate"],axis=1).dropna().shape, "||| différence : ", 12499-7525)
print("shape X_d colonnes 2 et 3 sans NaN : ",X_train_dropped[["Host Response Time","Host Response Rate"]].dropna().shape,"||| nombre de NaN dans col 2 et 3: ", 15692-8617)

shape X_train_Drop (X_d):  (15692, 31)
shape X_d sans NaN :  (7525, 31)
shape X_d sans colonnes 2 et 3 ET sans NaN :  (12499, 29) ||| différence :  4974
shape X_d colonnes 2 et 3 sans NaN :  (8617, 2) ||| nombre de NaN dans col 2 et 3:  7075


In [82]:
X_train_wo_star=X_train_dropped.loc[list_bool_star]
X_train_w_only_star=X_train_dropped.loc[switch_list_bool(  list_bool_star)]
# X_train_pro : on enlève les colonnes inutiles de X_train (X_train_dropped), puis toutes les lignes avec des étoiles (X_train_wo_star) puis on supprime les NaN
X_train_pro=X_train_pro_wo_star.dropna()

In [56]:
X_train_pro


,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,Postal Code,Latitude,Longitude,...,Last Review,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready
7,Alise,2015-04-30,within an hour,100%,f,Kreuzberg,Friedrichshain-Kreuzberg,10997,52.50021,13.43514,...,2019-04-23,94.0,9.0,9.0,10.0,10.0,10.0,10.0,t,f
9,Aselgiiud,2016-05-06,within an hour,100%,f,Moabit,Mitte,10553.0,52.53145,13.31710,...,2018-07-25,100.0,10.0,9.0,10.0,10.0,9.0,10.0,f,f
10,David,2013-06-09,within an hour,100%,f,Wilmersdorf,Charlottenburg-Wilm.,14197.0,52.47667,13.31001,...,2018-05-09,80.0,10.0,10.0,10.0,10.0,9.0,7.0,t,f
11,Nadon,2015-03-07,within a few hours,100%,t,NeukÃ¶lln,NeukÃ¶lln,12047.0,52.49554,13.42865,...,2019-02-24,99.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f
17,Max,2015-11-09,within an hour,83%,f,WeiÃensee,Pankow,13086.0,52.55110,13.45517,...,2018-08-22,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15678,Drmumde,2015-11-11,within a day,50%,f,Prenzlauer Berg,Pankow,10437.0,52.54058,13.42421,...,2018-07-19,75.0,8.0,5.0,7.0,7.0,10.0,8.0,f,f
15679,Maauddo,2015-05-04,within a day,100%,f,Friedrichshain,Friedrichshain-Kreuzberg,10245.0,52.50499,13.47396,...,2018-12-27,100.0,10.0,10.0,10.0,10.0,9.0,9.0,f,f
15682,Garissa,2013-04-16,within a day,88%,f,Kreuzberg,Friedrichshain-Kreuzberg,10997.0,52.50064,13.43893,...,2019-03-12,100.0,10.0,8.0,10.0,10.0,10.0,10.0,f,f
15687,Ascharl,2013-06-22,a few days or more,33%,f,Kreuzberg,Friedrichshain-Kreuzberg,10961.0,52.49016,13.39241,...,2019-01-02,98.0,10.0,9.0,10.0,10.0,10.0,10.0,f,f


NameError: name 'col' is not defined

In [36]:
c=0
for col in X_train.columns:
    if  "*" in X_train[col].values:
        print(col,X_train[col].value_counts())
        c+=1
print("Nombre de col avec *: ", c)

Listing Name *    15638
Name: Listing Name, dtype: int64
Host Response Rate 100%    6254
90%      419
80%      191
0%       185
50%      180
*        174
75%      111
67%      105
83%       89
70%       89
60%       80
88%       70
86%       64
89%       61
40%       54
94%       45
91%       41
33%       40
93%       39
92%       38
71%       33
96%       29
95%       28
99%       28
20%       26
78%       24
97%       21
98%       21
63%       19
82%       17
87%       13
25%       12
58%        9
30%        8
Name: Host Response Rate, dtype: int64
neighbourhood NeukÃ¶lln                2364
Prenzlauer Berg          2012
Kreuzberg                1946
Friedrichshain           1814
Mitte                    1403
                         ... 
Wannsee                    12
Friedrichshagen            12
Buckow                      9
Karow                       9
FranzÃ¶sisch Buchholz       6
Name: neighbourhood, Length: 63, dtype: int64
City Berlin    15644
*            47
Name: City, dtyp

C:\Users\Utlisateur\AppData\Local\Temp\ipykernel_11660\2078062795.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if  "*" in X_train[col].values:
C:\Users\Utlisateur\AppData\Local\Temp\ipykernel_11660\2078062795.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if  "*" in X_train[col].values:
C:\Users\Utlisateur\AppData\Local\Temp\ipykernel_11660\2078062795.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if  "*" in X_train[col].values:
C:\Users\Utlisateur\AppData\Local\Temp\ipykernel_11660\2078062795.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if  "*" in X_train[col].values:
C:\Users\Utlisateur\AppData\Local\Temp\ipykernel_11660\2078062795.py:3: 

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15692 entries, 0 to 15691
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Listing ID             15691 non-null  float64
 1   Listing Name           15638 non-null  object 
 2   Host ID                15692 non-null  float64
 3   Host Name              15670 non-null  object 
 4   Host Since             15671 non-null  object 
 5   Host Response Time     8617 non-null   object 
 6   Host Response Rate     8617 non-null   object 
 7   Is Superhost           15669 non-null  object 
 8   neighbourhood          15692 non-null  object 
 9   Neighborhood Group     15692 non-null  object 
 10  City                   15691 non-null  object 
 11  Postal Code            15463 non-null  object 
 12  Country Code           15692 non-null  object 
 13  Country                15692 non-null  object 
 14  Latitude               15692 non-null  float64
 15  Lo